# <div class='alert alert-info'>Ad type predictions</div>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
df= pd.read_csv('/Users/rajathadri_as/Documents/FAD - website_ads_ad - 20231130.csv')

In [ ]:
df.shape

In [ ]:
df=df[['title','type_name','type_id','cat_name','cat_id']]

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(subset=['title'], inplace=True)

In [ ]:
df['type_name'].value_counts()

In [ ]:
# Handling null values in Type_name
df.loc[df['type_id'] == 4, 'type_name'] = 'Services'
df.loc[df['type_id'] == 9, 'type_name'] = 'Adult Businesses'

In [ ]:
# Handling null values in Cat_name
df.loc[df['cat_id'] == 190, 'cat_name'] = 'Business & Office'
df.loc[df['cat_id'] == 191, 'cat_name'] = 'Professional Services'
df.loc[df['cat_id'] == 192, 'cat_name'] = 'Events & Celebrations'
df.loc[df['cat_id'] == 193, 'cat_name'] = 'Family & Care Services'
df.loc[df['cat_id'] == 194, 'cat_name'] = 'Health, Beauty & Wellness'
df.loc[df['cat_id'] == 195, 'cat_name'] = 'Fitness'
df.loc[df['cat_id'] == 196, 'cat_name'] = 'Lessons & Tuition'
df.loc[df['cat_id'] == 197, 'cat_name'] = 'Home & Garden'
df.loc[df['cat_id'] == 198, 'cat_name'] = 'Pets & Horses'
df.loc[df['cat_id'] == 199, 'cat_name'] = 'Motoring & Transport'
df.loc[df['cat_id'] == 201, 'cat_name'] = 'Local Food & Drink'
df.loc[df['cat_id'] == 202, 'cat_name'] = 'Retail'
df.loc[df['cat_id'] == 263, 'cat_name'] = 'Escort Agencies'

In [ ]:
df['type_name'].value_counts()

In [ ]:
df = df[df['cat_id']!=0]

In [ ]:
df = df[df['type_id']!=0]

In [ ]:
df = df[df['type_name'] != 'Services']

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df.shape

In [ ]:
df['type_name'].value_counts()

In [ ]:
df.head()

In [ ]:
df['type_name'].value_counts(normalize = True).plot.bar()

In [ ]:
df['cat_name'].value_counts(normalize = True).plot.bar()

# <div class = 'alert alert-info'>NLTK text processing</div>

In [ ]:
import nltk
nltk.download('all')

In [ ]:
# create a list text
text = list(df['title'])

In [ ]:
# preprocessing loop
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
corpus = []

for i in range(len(text)):
    r = re.sub('[^a-zA-Z]', ' ', text[i])
    r = r.lower()
    r = r.split()
    r = [word for word in r if word not in stopwords.words('english')]
    r = [lemmatizer.lemmatize(word) for word in r]
    r = ' '.join(r)
    corpus.append(r)

In [ ]:
corpus

In [ ]:
#assign corpus to df
df['title'] = corpus
df.head()

In [ ]:
def lemmatize_words(text):
    words = text.split()
    words = [lemmatizer.lemmatize(word,pos='v') for word in words]
    return ' '.join(words)

In [ ]:
df['title'] = df['title'].apply(lemmatize_words)

In [ ]:
df.head()

In [ ]:
# import these modules
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

ps = PorterStemmer()

In [ ]:
df['title'] = df['title'].apply(ps.stem)

In [ ]:
df[['type_id','type_name']].value_counts()

In [ ]:
df.head()

# <div class = 'alert alert-info'>Feature Engineering</div>

In [ ]:
df.head(3)

### Null Check

In [ ]:
df.isnull().sum()

### Outliers

In [ ]:
df.describe()

In [ ]:
fig = plt.figure(figsize =(5, 5))

plt.boxplot(df['type_id'])
plt.show()

In [ ]:
fig = plt.figure(figsize =(5, 5))

plt.boxplot(df['cat_id'])
plt.show()

In [ ]:
df['cat_id'].unique()

In [ ]:
df['cat_id'].value_counts()

In [ ]:
# 292, 295, 298, 300

In [ ]:
# df = df[df['cat_id'] < 290]

# <div class = 'alert alert-info'>Modeling - Ad Type</div>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

---

In [ ]:
X = df['title']
y = df[['type_name']]

# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# X_scaled = scaler.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cv = CountVectorizer()

X_train_cv = cv.fit_transform(X_train)

X_train_cv.shape

In [ ]:
# transform X_test using CV
X_test_cv = cv.transform(X_test)

X_test_cv.shape

In [ ]:
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')

In [ ]:
X_train_cv = tfidfvectorizer.fit_transform(X_train)

X_train_cv.shape

In [ ]:
X_test_cv = tfidfvectorizer.transform(X_test)

X_test_cv.shape

In [ ]:
Break Auto Execution # Self-induced error

---

# <div class = 'alert alert-warning'> Logistic Regression Classifier</div>

In [ ]:
# Logistic Regression model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV


lr = LogisticRegression()

lr.fit(X_train_cv, y_train)

In [ ]:
# Predictions
y_pred_test = lr.predict(X_test_cv)
y_pred_test

In [ ]:
y_pred_train = lr.predict(X_train_cv)
y_pred_train

In [ ]:
print(classification_report(y_pred_train, y_train))

In [ ]:
print(classification_report(y_pred_test, y_test))

In [ ]:
cf_matrix = confusion_matrix(y_train, y_pred_train)

print(cf_matrix)

In [ ]:
cf_matrix = confusion_matrix(y_test, y_pred_test)

print(cf_matrix)

# <div class = 'alert alert-success'>CV for Logistic Regression</div>

In [ ]:
lr = LogisticRegression()

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 200]}
grid = GridSearchCV(lr, param_grid, cv=3)
grid.fit(X_train_cv, y_train)


# Recheck the best cross-validation sccore

print("Best cross-validation score:", grid.best_score_)
print("Best parameters:", grid.best_params_)
print("Test set score:", grid.score(X_test_cv, y_test))

In [ ]:
y_pred_test = grid.predict(X_test_cv)

In [ ]:
y_pred_train = grid.predict(X_train_cv)

In [ ]:
print(classification_report(y_train, y_pred_train))

In [ ]:
print("Classification report for test data: \n")
print(classification_report(y_test, y_pred_test))

In [ ]:
cf_matrix = confusion_matrix(y_test, y_pred_test)

print(cf_matrix)

# <div class = 'alert alert-warning'>Random Forest Classifer</div>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
clsr = RandomForestClassifier(n_estimators=100)
clsr.fit(X_train_cv, y_train)

In [ ]:
y_pred_train = clsr.predict(X_train_cv)
train_score = accuracy_score(y_train, y_pred_train)
train_score

In [ ]:
# Predictions
y_pred_test = clsr.predict(X_test_cv)
test_score= accuracy_score(y_test, y_pred_test)
test_score

In [ ]:
# Classification Report
print('Classification report for Random Forest (Train set) = \n')
print(classification_report(y_pred_train, y_train))

In [ ]:
# Classification Report
print('Classification report for Random Forest (Test set)= \n')
print(classification_report(y_pred_test, y_test))

# <div class = 'alert alert-warning'>CV for Random Forest</div>

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 0, stop = 400, num = 10)]

# Number of features to consider at every split
max_features = ['sqrt','log2']

# Maximum number of levels in tree
max_depth = [2,4]

# Minimum number of samples required to split a node
min_samples_split = [2, 5]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]

# Method of selecting samples for training each tree
bootstrap = [True, False]

In [ ]:
# Create the param grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(param_grid)

In [ ]:
rf = RandomForestClassifier()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

rf_RandomGrid = RandomizedSearchCV(estimator = rf, param_distributions = param_grid, scoring='f1', cv = 10, verbose=2, n_jobs = 4)

In [ ]:
rf_RandomGrid.fit(X_train_cv, y_train)

In [ ]:
rf_RandomGrid.best_params_

In [ ]:
confusion_matrix(y_test, y_pred_test)

In [ ]:
print(classification_report(y_test, y_pred_test))

In [ ]:
# params = {'n_estimators':[200],
#           'max_features':['sqrt',4,'log2','auto'],
#           'max_leaf_nodes':[10, 20, 40]
#           }

# # Apply random Forest Classifier Model

# rf_cv = GridSearchCV(rf, params, scoring='f1', cv=5)
# rf_cv.fit(X_train_cv, y_train)

In [ ]:
# rf_cv.best_params_

In [ ]:
# rf_cv.best_estimator_

In [ ]:
# rf_cv.best_score_

In [ ]:
# rf_cv = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
#                        criterion='gini', max_depth=40, max_features='sqrt',
#                        max_leaf_nodes=40, max_samples=None,
#                        min_impurity_decrease=0.0,
#                        min_samples_leaf=1, min_samples_split=2,
#                        min_weight_fraction_leaf=0.0, n_estimators=200,
#                        n_jobs=None, oob_score=False, random_state=None,
#                        verbose=0, warm_start=False)
# rf_cv.fit(X_train_cv, y_train)

In [ ]:
# y_pred_train = rf_cv.predict(X_train_cv)
# accuracy_score(y_train, y_pred_train)

In [ ]:
# y_pred_test = rf_cv.predict(X_test_cv)
# accuracy_score(y_test, y_pred_test)

# <div class = 'alert alert-warning'>XGBoost Classifier</div>

In [ ]:
# Already installed

# pip install xgboost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
df2 = df.copy()

In [ ]:
df2.loc[df['type_id'] == 1, 'type_id'] = 0
df2.loc[df['type_id'] == 2, 'type_id'] = 1
df2.loc[df['type_id'] == 3, 'type_id'] = 2
df2.loc[df['type_id'] == 5, 'type_id'] = 3
df2.loc[df['type_id'] == 6, 'type_id'] = 4
df2.loc[df['type_id'] == 7, 'type_id'] = 5
df2.loc[df['type_id'] == 8, 'type_id'] = 6
df2.loc[df['type_id'] == 9, 'type_id'] = 7

In [ ]:
df2[['type_id','type_name']].value_counts()

In [ ]:
X = df2['title']
y = df2[['type_id']]


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()

X_train_cv = cv.fit_transform(X_train)

X_train_cv.shape

In [ ]:
# transform X_test using CV
X_test_cv = cv.transform(X_test)

X_test_cv.shape

In [ ]:
xgb = XGBClassifier(max_depth = 5, learning_rate = 0.1)
xgb.fit(X_train_cv, y_train)
XGBClassifier(max_depth=5, objective='multi:softprob')

In [ ]:
# Predictions
y_pred_train = xgb.predict(X_train_cv)
y_pred_test = xgb.predict(X_test_cv)

In [ ]:
score = classification_report(y_train, y_pred_train)
print('Classification Report for XGBoost(Train set)= \n')
print(score)

In [ ]:
score = classification_report(y_test, y_pred_test)
print('Classification Report for XGBoost(Train set)= \n')
print(score)

# <div class = 'alert alert-success'>Pickle and Predict</div>

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [ ]:
pipe_lr = Pipeline(steps=[('cv',TfidfVectorizer(analyzer='word',stop_words= 'english')),
                          ('lr_multi',MultiOutputClassifier(
                              GridSearchCV(LogisticRegression(), {'C': [0.001, 0.01, 0.1, 1, 10, 100, 200]}, cv=5)
                              ))]
                  )

In [ ]:
pipe_lr.fit(X_train, y_train)

In [ ]:
pipe_lr.score(X_train,y_train)

In [ ]:
pipe_lr.score(X_test,y_test)

In [ ]:
# Save the model to a file using pickle
filename = 'FAD_MLclassifier_AdType.pkl'
with open(filename, 'wb') as file:
    pickle.dump(pipe_lr, file)

In [ ]:
file.close()

In [1]:
# Let's Predict

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pickle

In [2]:
with open('FAD_AdTypeClassifier_model.pkl', 'rb') as file:
    mod = pickle.load(file)

# Load some new data for prediction
X_new = ["Hiring for Services"]

In [ ]:
mod.predict(X_new)

In [ ]:
file.close()

In [ ]:
# tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')

# X_train_new = tfidfvectorizer.fit_transform(X_train)
# X_new_cv = tfidfvectorizer.transform(X_new)

# X_new_cv.shape

In [ ]:
# print(X_new_cv)

In [3]:
# # Predict the output for the new data using the loaded model
# prediction = mod.predict(X_new)

# print(prediction)

ValueError: Expected 2D array, got 1D array instead:
array=['Hiring for Services'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.